In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle
import base64
from email.mime.text import MIMEText
from email.utils import formataddr, parseaddr

# Scopes for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def create_reply_message(service, original_message, reply_body):
    """Create a reply to an email message."""
    # Extract necessary details from the original message
    thread_id = original_message['threadId']
    message_id = None
    headers = original_message['payload'].get('headers', [])
    for header in headers:
        if header['name'] == 'Message-ID':
            message_id = header['value']
        elif header['name'] == 'From':
            sender = header['value']
        elif header['name'] == 'To':
            recipient = header['value']
        elif header['name'] == 'Subject':
            subject = header['value']

    # Create the reply email
    reply = MIMEText(reply_body)
    reply['To'] = sender  # Reply to the original sender
    reply['From'] = recipient  # Your email address
    reply['Subject'] = f"Re: {subject}"
    reply['In-Reply-To'] = message_id
    reply['References'] = message_id

    # Encode the message
    raw_message = base64.urlsafe_b64encode(reply.as_bytes()).decode()

    # Create the message object with threadId
    return {
        'raw': raw_message,
        'threadId': thread_id
    }

def send_reply(service, original_message, reply_body):
    """Send a reply to an email message."""
    reply = create_reply_message(service, original_message, reply_body)
    try:
        sent_message = service.users().messages().send(userId='me', body=reply).execute()
        print(f"Reply sent to thread ID: {sent_message['threadId']}")
    except Exception as error:
        print(f"An error occurred: {error}")

def main():
    creds = None
    # Load credentials from token.pickle if it exists
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If no credentials, authenticate the user
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Access Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # Get the list of messages
    results = service.users().messages().list(userId='me', maxResults=10).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
    else:
        # Process the first message as an example
        message = messages[0]
        msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()

        # Define your reply body
        reply_body = "Thank you for your email. This is a reply."

        # Send the reply
        send_reply(service, msg, reply_body)

if __name__ == '__main__':
    main()


In [ ]:
import os
import pickle
import base64
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Define the scope for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    """Authenticate the user and return the Gmail service."""
    creds = None
    # Load credentials from token.pickle if it exists
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    # Build the Gmail service
    service = build('gmail', 'v1', credentials=creds)
    return service

def get_latest_email(service):
    """Retrieve the most recent email from the user's inbox."""
    # Fetch the latest email
    results = service.users().messages().list(userId='me', maxResults=1, q="in:inbox").execute()
    messages = results.get('messages', [])
    if not messages:
        print("No messages found.")
        return None
    # Get the message details
    message = service.users().messages().get(userId='me', id=messages[0]['id'], format='full').execute()
    print(f"Latest email ID: {message['id']}")
    return message

def create_reply_message(service, original_message, reply_body):
    """Create a reply message for the given original email."""
    # Extract necessary details from the original message
    thread_id = original_message['threadId']
    message_id = None
    headers = original_message['payload'].get('headers', [])
    for header in headers:
        if header['name'] == 'Message-ID':
            message_id = header['value']
        elif header['name'] == 'From':
            sender = header['value']
        elif header['name'] == 'To':
            recipient = header['value']
        elif header['name'] == 'Subject':
            subject = header['value']

    # Create the reply email
    reply = MIMEText(reply_body)
    reply['To'] = sender  # Reply to the original sender
    reply['From'] = recipient  # Your email address
    reply['Subject'] = f"Re: {subject}"
    reply['In-Reply-To'] = message_id
    reply['References'] = message_id

    # Encode the message
    raw_message = base64.urlsafe_b64encode(reply.as_bytes()).decode()

    # Create the message object with threadId
    return {
        'raw': raw_message,
        'threadId': thread_id
    }

def send_reply(service, original_message, reply_body):
    """Send a reply to the original email."""
    reply = create_reply_message(service, original_message, reply_body)
    try:
        sent_message = service.users().messages().send(userId='me', body=reply).execute()
        print(f"Reply sent to thread ID: {sent_message['threadId']}")
    except Exception as error:
        print(f"An error occurred: {error}")

def main():
    # Authenticate and build the Gmail service
    service = authenticate_gmail()

    # Get the latest email
    latest_email = get_latest_email(service)
    if not latest_email:
        return

    # Define your reply body
    reply_body = "Thank you for your email. This is an automated response."

    # Send the reply
    send_reply(service, latest_email, reply_body)

if __name__ == '__main__':
    main()

In [6]:
import os
import pickle
import base64
import json
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Define the scope for Gmail API
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.send'
]

def authenticate_gmail():
    """Authenticate the user and return the Gmail service."""
    creds = None
    # Load credentials from token.pickle if it exists
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0, access_type='offline', prompt='consent')
        # Save the credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    # Build the Gmail service
    service = build('gmail', 'v1', credentials=creds)
    return service

def get_latest_email(service):
    """Retrieve the most recent email from the user's Primary inbox."""
    try:
        # Fetch the latest email from the Primary category
        results = service.users().messages().list(
            userId='me',
            maxResults=1,
            q="category:primary"
        ).execute()
        messages = results.get('messages', [])
        if not messages:
            print("No messages found in the Primary category.")
            return None
        # Get the message details
        message = service.users().messages().get(
            userId='me',
            id=messages[0]['id'],
            format='full'
        ).execute()
        return message
    except Exception as error:
        print(f"An error occurred while fetching the latest email: {error}")
        return None

def decode_message_body(encoded_body):
    """Decode the base64url-encoded message body."""
    decoded_bytes = base64.urlsafe_b64decode(encoded_body)
    return decoded_bytes.decode('utf-8')

def extract_email_details(message):
    """Extract details from the email message."""
    headers = message['payload'].get('headers', [])
    subject = next((header['value'] for header in headers if header['name'] == 'Subject'), None)
    sender = next((header['value'] for header in headers if header['name'] == 'From'), None)
    recipient = next((header['value'] for header in headers if header['name'] == 'To'), None)
    body = ""
    if 'parts' in message['payload']:
        for part in message['payload']['parts']:
            if part['mimeType'] == 'text/plain' and 'data' in part['body']:
                body = decode_message_body(part['body']['data'])
                break
    elif 'body' in message['payload'] and 'data' in message['payload']['body']:
        body = decode_message_body(message['payload']['body']['data'])
    return {
        "Subject": subject,
        "From": sender,
        "To": recipient,
        "Body": body
    }

def create_reply_message(service, original_message, reply_body):
    """Create a reply message for the given original email."""
    try:
        # Extract necessary details from the original message
        thread_id = original_message['threadId']
        message_id = None
        headers = original_message['payload'].get('headers', [])
        for header in headers:
            if header['name'] == 'Message-ID':
                message_id = header['value']
            elif header['name'] == 'From':
                sender = header['value']
            elif header['name'] == 'To':
                recipient = header['value']
            elif header['name'] == 'Subject':
                subject = header['value']

        # Create the reply email
        reply = MIMEText(reply_body)
        reply['To'] = sender  # Reply to the original sender
        reply['From'] = recipient  # Your email address
        reply['Subject'] = f"Re: {subject}"
        reply['In-Reply-To'] = message_id
        reply['References'] = message_id

        # Encode the message
        raw_message = base64.urlsafe_b64encode(reply.as_bytes()).decode()

        # Create the message object with threadId
        return {
            'raw': raw_message,
            'threadId': thread_id
        }
    except Exception as error:
        print(f"An error occurred while creating the reply message: {error}")
        return None

def send_reply(service, original_message, reply_body):
    """Send a reply to an email message."""
    try:
        reply = create_reply_message(service, original_message, reply_body)
        if reply:
            sent_message = service.users().messages().send(userId='me', body=reply).execute()
            print(f"Reply sent to thread ID: {sent_message['threadId']}")
        else:
            print("Failed to create reply message.")
    except Exception as error:
        print(f"An error occurred while sending the reply: {error}")

def main():
    # Authenticate and build the Gmail service
    service = authenticate_gmail()

    # Fetch the latest email
    latest_email = get_latest_email(service)
    if latest_email:
        # Extract email details
        email_details = extract_email_details(latest_email)
        # Print email details in JSON format
        print(json.dumps(email_details, indent=4))

        # Define your reply body
        reply_body = "Thank you for your email. This is an automated response."

        # Send the reply
        send_reply(service, latest_email, reply_body)

if __name__ == '__main__':
    main()

{
    "Subject": "Test",
    "From": "Karan Vora <kv2154@nyu.edu>",
    "To": "Karan Vora <karanvora210@gmail.com>",
    "Body": "This is a test email\r\n\r\nBest Regards,\r\nKaran Vora\r\nNew York University Class of 2024\r\nMS Computer Engineering\r\nkv2154@nyu.edu\r\nkaranvora210@gmail.com\r\nContact Number: +1 929-930-0027\r\nLinkedin <https://www.linkedin.com/in/karan-vora-574961188/>\r\n"
}
Reply sent to thread ID: 1944c1befe70967b
